### Integração Numérica - Regra do Trapézio

Este notebook apresenta a implementação da Regra do Trapézio para cálculo de integrais definidas, incluindo o erro de aproximação simples e o erro generalizado.

In [1]:
%load_ext autoreload
%autoreload 2
from integral import Integral
import math
import numpy as np

### Classe `IntegralTrapezio`

In [2]:
class IntegralTrapezio(Integral):
    def __init__(self, x=None, y=None, limite_inf=None, limite_sup=None, func_integral=None, h=None):
        super().__init__()
        self._setup(x, y, limite_inf, limite_sup, func_integral, h)

    def integral(self):
        # regra do trapézio simples sobre todos os pontos
        soma = self.y[0] + self.y[-1] + 2 * np.sum(self.y[1:-1])
        return (self.h / 2) * soma

    def erro_simples(self):
        d2_inf = self.interpolador.derivative(self.limite_inf, 2)
        d2_sup = self.interpolador.derivative(self.limite_sup, 2)
        maior = self.saber_maior(d2_inf, d2_sup)
        parcela_1 = ((self.limite_sup - self.limite_inf) * (self.h**2)) / 12
        return abs(maior * parcela_1)

    def erro_generalizado(self):
        # No trapézio geralmente erro é dado pela segunda derivada (2ª ordem)
        return self.erro_simples()

### Exemplo de uso com pontos

In [3]:
# Célula 1 - Integral Trapézio com pontos (x, y)
def f(x):
    return x * math.sqrt(x**2 + 1)

pontos_x = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
pontos_y = [f(xi) for xi in pontos_x]

trap = IntegralTrapezio(x=pontos_x, y=pontos_y)
print("Trapézio (x, y):", trap.integral())
print("Erro simples:", trap.erro_simples())
print("Erro generalizado:", trap.erro_generalizado())


Trapézio (x, y): 0.1327990413780471
Erro simples: 0.0005202933133542894
Erro generalizado: 0.0005202933133542894


### Exemplo de uso com função

In [4]:
# Célula 2 - Integral Trapézio com função, limites e h
def f(x):
    return x * math.sqrt(x**2 + 1)

trap2 = IntegralTrapezio(limite_inf=0, limite_sup=0.5, func_integral=f, h=0.1)
print("Trapézio (função):", trap2.integral())
print("Erro simples:", trap2.erro_simples())
print("Erro generalizado:", trap2.erro_generalizado())


Trapézio (função): 0.1327990413780471
Erro simples: 0.0005202933133542858
Erro generalizado: 0.0005202933133542858
